**Install and import dependencies**

In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install transformers requests beautifulsoup4 pandas numpy

In [30]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

**Instantiate model**

In [4]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

**Collect reviews**

In [22]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [31]:
reviews[0]

'Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coffee was good as well'

**Load reviews into dataframe and score**

In [36]:
df = pd.DataFrame(reviews,columns = ["reviews"])
df

,reviews
0,Great staff and food. Must try is the pan fri...
1,i have never been to this place nor heard of i...
2,i have never been there but the food is horrible
3,It was ok. The coffee wasn't the best but it w...
4,I came to Social brew cafe for brunch while ex...
5,Ricotta hot cakes! These were so yummy. I ate ...
6,I went here a little while ago- a beautiful mo...
7,We came for brunch twice in our week-long visi...
8,Ron & Jo are on the go down under and Wow! We...
9,Great coffee and vibe. That's all you need. C...


In [61]:
def sentiment_score(review):
  tokens = tokenizer.encode(review,return_tensors = 'pt',max_length = 512,truncation=True)
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [62]:
sentiment_score(df["reviews"].iloc[1])

1

In [65]:
for i in range(0,len(df)):
  print(df["reviews"].iloc[i],":","Score",sentiment_score(df["reviews"].iloc[i]))

Great staff and food.  Must try is the pan fried Gnocchi!  The staff were really friendly and the coffee was good as well : Score 5
i have never been to this place nor heard of it. i have heard that the food is disgusting from @Savelii P., the restaraunt provides no free soy sauce therefore it is considered garbage. 1/5 stars max : Score 1
i have never been there but the food is horrible : Score 1
It was ok. The coffee wasn't the best but it was fine. The relish on the breakfast roll was yum which did make it sing. So perhaps I just got a bad coffee but the food was good on my visit. : Score 3
I came to Social brew cafe for brunch while exploring the city and on my way to the aquarium. I sat outside. The service was great and the food was good too!I ordered smoked salmon, truffle fries, black coffee and beer. : Score 5
Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all 

In [67]:
df["sentiment"] = [sentiment_score(df["reviews"].iloc[i]) for i in range(0,len(df))]

In [68]:
df

,reviews,sentiment
0,Great staff and food. Must try is the pan fri...,5
1,i have never been to this place nor heard of i...,1
2,i have never been there but the food is horrible,1
3,It was ok. The coffee wasn't the best but it w...,3
4,I came to Social brew cafe for brunch while ex...,5
5,Ricotta hot cakes! These were so yummy. I ate ...,5
6,I went here a little while ago- a beautiful mo...,1
7,We came for brunch twice in our week-long visi...,5
8,Ron & Jo are on the go down under and Wow! We...,5
9,Great coffee and vibe. That's all you need. C...,5
